# Планирование задач. Введение в Apache Airflow

Обычно развитие data—driven инициатив начинается с ручного управления задачами. Например, для машинного обучения это будут процессы подготовки датасета, обучения моделей, отслеживание результатов и развертывание решений на рабочем сервере (production). Эти процедуры растут и усложняются по мере увеличения команды и продвижения самого продукта. В частности, растет количество повторяющихся шагов, повышается зависимость задач друг от друга и степень их важности для бизнеса. Таким образом, появляется целый конвейер (__pipeline__) задач, которые нужно запускать с некоторой периодичностью в определенном порядке.

<img src="images/1/data-pipeline-schema.jpg" style="width: 500px;"> 

Тоже самое касается и задач, связанных с обработкой данных, когда в определенный момент времени нужно собрать свежие данные, обработать их и выполнить с ними какие-то операции - собрать витрину и сохранить в БД, обучить модель ML, подготовить отчет в Excel и разослать по e-mail - вариантов множество. Для эффективного решения таких задач требуются соответствующие инструменты.

## Планировщик задач cron

В курсе по Linux вы уже знакомились с таким инструментом, как [__cron__](https://ru.wikipedia.org/wiki/Cron) - демоном (компьютерная программа в UNIX-системах, запускаемая самой системой и работающая в фоновом режиме без прямого взаимодействия с пользователем), использующимся для периодического выполнения заданий в определённое время. С его помощью вы можете ставить на расписание в __crontab__ bash-скрипты, python-скрипты, запускать Jupyter-ноутбуки и много другое. Вспомнить формат cron'а можно [здесь](http://www.nncron.ru/nncronlt/help/RU/working/cron-format.htm), а проверить свое cron-расписание можно [тут](https://crontab.guru/).

<img src="images/1/cron.png" style="width: 500px;"> 

Cron отлично справляется с простыми периодическими процессами, однако, все становится хуже, когда вы начинаете поддерживать рабочий процесс в крупных проектах, где между задачами существуют зависимости. Например, нужно запустить задачу C, как только выполнятся задачи A и B, причем так как задача A может упасть с ошибкой, то хотим дать ей 3 попытки на успешное выполнение. Также очевидна проблема масштабируемости: если 2-3 задачи легко видеть и настраивать в cron, то что если задач будет 100? И так далее.

Итого, среди основных недостатков cron можно выделить следующие:
* отсутствие масштабируемости 
* нельзя прописать зависимости между задачами
* нет мониторинга
* отсутствует механизим повторного запуска
* не разделяет идею "fault tolerance"

Для удобной организации большого кол-ва процессов используют специальные фреймворки, к самым популярным из которых относятся __Airflow__ от Airbnb и __Luigi__ от Spotify. Рассмотрим Airflow более подробно, а в следующих уроках сравним его с Luigi.

## Apache Airflow

<img src="images/1/AirflowLogo.png" style="width: 300px;"> 

Apache Airflow — это open-source набор библиотек для разработки, планирования и мониторинга рабочих процессов, который позволяет создавать и настраивать цепочки задач как в визуальном режиме с помощью наглядного web-GUI, так и писать программный код Python. Разработан в 2014 году в компании Airbnb, автор Maxime Beauchemin, через 2 года AirFlow был передан в фонд Apache Software Foundation. С 2019 года этот фреймворк официально стал проектом Apache 1-го уровня (Top-Level).

17 декабря 2020 была [анонсирована](http://airflow.apache.org/blog/airflow-two-point-oh-is-here/) долгожданная версия 2.0.

### Почему именно Airflow?

Вот несколько причин использовать Apache Airflow:

* __Открытый исходный код__: после запуска в качестве внутреннего проекта Airbnb, Airflow естественным образом нуждался в сообществе. Это было основной причиной, почему он в конечном итоге стал проектом с открытым исходным кодом. В настоящее время поддерживается и управляется как инкубаторный проект на в Apache.

* __Веб-интерфейс__: Airflow ставится с Flaskapp, который отслеживает все рабочие процессы и позволяет вам легко изменять, запускать или останавливать их. Вы также можете работать с командной строкой, но веб-интерфейс более интуитивно понятен, он обеспечивает относительно низкий порог входа в технологию, позволяя работать с Airflow не только инженеру данных (Data Engineer), но и аналитику, разработчику, администратору и DevOps-инженеру.

* __На основе Python__: каждая часть конфигурации написана на языке Python, включая настройку расписаний и скриптов для их запуска. Это устраняет необходимость использовать JSON или XML конфигурационные файлы. К тому же Python - стандарт де-факто для современного специалиста в области Big Data и Data Science: инженера, аналитика, разработчика больших данных и специалиста по машинному обучению.

* __Широкое использование__: Airflow используют такие компании, как Airbnb, Intel, PayPal, WePay, Yahoo!, это незаменимый инструмент в арсенале современного дата инженера, если смотреть открытые вакансии на позицию data engineer, то нередко встретишь опыт работы с Airflow как одно из требований к позиции. 

* __Удобен в применении__: несложная инсталяция и первые шаги, хорошая визуализация, возможность автоматически создавать большое число задач и широкие возможности кастомизации.

* __Легко масштабируем__: модульной архитектуры и очереди сообщений для неограниченного числа DAG’ов (Celery/Dask).

* __Собственный репозиторий метаданных__: на базе библиотеки SqlAlchemy, где хранятся состояния задач, DAG’ов, глобальные переменные и пр.

* __Интеграция со множеством источников и сервисов__: базы данных (MySQL, PostgreSQL, DynamoDB, Hive), Big Data хранилища (HDFS, Amazon S3) и облачные платформ (Google Cloud Platform, Amazon Web Services, Microsoft Azure).

* __Расширяемый REST API__: его относительно легко интегрировать Airflow в существующий ИТ-ландшафт корпоративной инфраструктуры и гибко настраивать конвейеры данных, например, передавать POST-параметры в DAG.

AirFlow принято называть ETL-средством для пакетов (батчей) Big Data, он не является классической ETL-системой, а лишь помогает представить процесс извлечения-преобразования-загрузки данных в виде единого проекта на Python, чтобы удобно и эффективно управлять им.

На практике Apache Airflow используется в следующих случаях:

* интеграция множества информационных систем и данных из различных источников (внутренние и внешние базы, файловые хранилища, облачные приложения и пр.);
* загрузка информации в корпоративное озеро данных (Data Lake);
* организация уникальных конвейеров доставки и обработки больших данных (data pipeline);
* управление конфигурацией конвейеров данных как кодом в соответствии с DevOps-подходом;
* автоматизация разработки, планирования и мониторинга batch-процессов обработки Data Flow.

### Основные понятия в Airflow

---
* __Направленный ациклический граф - DAG (Directed Acyclic Graph)__

<img src="images/1/graph.png" style="width: 500px;"> 

В Apache Airflow для создания рабочих потоков используются направленные ациклические графы (DAG), т.е. это такой граф, у которого отсутствуют циклы, но могут быть параллельные пути, выходящие из одного и того же узла. DAG — это набор задач конвейера данных, которые надо выполнить в строго определенной последовательности по определенному расписанию в рамках единой смысловой цепочки (расписание задается в формате __cron__). Можно устанавливать зависимости не только внутри одного DAG, но и между несколькими DAG’ами. Часть задач можно пропускать (skip) в зависимости от условий, статусов завершения предыдущих задач и т.п. Также для задач можно выставлять различные приоритеты (priority_weight).

На картинке ниже можно видеть классический DAG, где Task E является конечным в цепочке и зависит от всех задача слева от него.

<img src="images/1/dag.png" style="width: 400px;"> 

Например, DAG, который считывает данные из 3 источников независимо друг от друга. После этого мы запускаем задание Spark для соединения данных по ключу и записываем результат преобразования в Redshift. Определение DAG позволяет планировщику понять, какие задачи могут быть запущены немедленно, а какие должны ждать завершения других задач. Задание Spark должно дождаться трех задач «чтения» и заполнить данные в S3 и HDFS.

<img src="images/1/airflow_case_example.png" style="width: 500px;"> 

__Важно!__ Созданный в Airflow DAG идемпотентен - при повторных запусках дает один и тот же результат. Это реализовано с помощью  переменной execution_date - переменная каждого запуска пайплайна.

В Airflow WebUI DAG'и наглядно визуализируются даже большие и сложные задачи:

<img src="images/1/big_dag.png" style="width: 600px;"> 

DAG в Airflow может состоять из множества веток, различных ветвлений и т.п. 

---
* __Планировщик (Scheduler)__

Планировщик — это мозг настройки рабочих процессов в Airflow. Для пользователя взаимодействие с планировщиком ограничивается предоставлением ему информации о различных задачах и когда они должны запускаться.

---
* __Операторы (Operators)__

Операторы — это «рабочие», которые выполняют наши задачи, это звено в цепочке задач. Используя оператор разработчик описывает какую задачу необходимо выполнить. Рабочие процессы определяются путем создания группы DAG операторов. Каждый оператор выполняет определенную задачу, написанную в виде функции на Python или через shell-скрипты. Вы можете создавать собственные пользовательские операторы, расширяя класс BaseOperator и реализуя этот execute() метод (подробнее [тут](https://airflow.apache.org/docs/apache-airflow/stable/howto/custom-operator.html)).

В Airflow есть ряд готовых операторов, например:

* _PythonOperator_ — оператор для исполнения python кода
* _BashOperator_ — оператор для запуска bash скриптов/команд
* _PostgresOperator_ — оператор для вызова SQL запросов в PostgreSQL БД, аналогичные есть для MySQL, Oracle, Hive и т.д.
* _RedshiftToS3Transfer_ — оператор для запуска UNLOAD команды из Redshift в S3
* _EmailOperator_ — оператор для отправки электронных писем
* _DummyOperator_ —  выполняет роль пустышки и создан для того, чтобы склеивать различные участки пайплайна между собой
* _DockerOpetator_ — запускает Docker-контейнер на воркере, нужно понимать, что внутри Docker-контейнера может запуститься все, что угодно, поэтому очень важно при этом мониторить ресурсы воркера, чтобы они внезапно не закончились
* _KubernetesPodOperator_ — запускает новый pod в Kubernetes

Полный список стандартных операторов можно найти в [документации Apache Airflow](https://airflow.apache.org/docs/apache-airflow/stable/index.html).

DAG является объединяющей сущностью для набора операторов, т.е. если вернуться к картинке выше, то Task A, Task B и т.д. это отдельные операторы.

__Важно!__ Операторы не могут принимать возвращаемые значения от выполнения предыдущих операторов в цепочке (как, например, цепочка из вызовов функций), т.к. могут исполняться в разном адресном пространстве и даже на разных физических машинах.

---
* __Задачи (Tasks)__

Задачи — это определяемые пользователем действия, выполняемые операторами. Они могут быть функциями в Python или внешними скриптами, которые вы можете легко вызвать. Задачи должны быть написаны так, чтобы независимо от того, сколько раз вы запускаете эту задачу, это должно привести к тому же результату для тех же входных параметров.

<img src="images/1/dag_logic.png" style="width: 400px;"> 

__Важно!__ Не путайте операторы и задачи. Задачи определяют «что запускать?», а операторы — «как запустить?».  Например, функция на Python, которая считывает данные из S3 и записывает в базу данных — это задача. Метод, который вызывает эту функцию на Python в Airflow — это оператор. Airflow имеет встроенные операторы, которые вы можете использовать для общих задач.

---
* __Сенсор (Sensor)__

Сенсор - это разновидность Operator, его удобно использовать при реализации событийно ориентированных (event-driven) пайплайнов, например, нужно дождаться нового файла или готовности отчета по http. Из стандартного набора есть, например:

* _PythonSensor_ — ждём, когда функция вернёт True
* _S3Sensor_ — проверяет наличие объекта по ключу в S3-бакете
* _RedisPubSubSensor_ — проверяет наличие сообщения в pub-sub очереди
* _RedisKeySensor_ — проверяет существует ли переданный ключ в Redis хранилище

Это лишь малая часть доступных для использования сенсоров. Чтобы создать свой сенсор, достаточно унаследоваться от BaseSensorOperator и переопределить метод poke.

---
* __Экзекьюторы (Executors)__

Executors отвечают за исполнение задач (tasks), в Airflow есть несколько видов исполнителей:

* _SequentialExecutor_ — этот исполнитель установлен в качестве значения по умолчанию в airflow.cfg у параметра executor и представляет из себя простой вид исполнителя, который не умеет запускать параллельные задачи. Как можно догадаться, в конкретный момент времени выполняться может только одна единственная задача. Этот вид исполнителя используют в ознакомительных целях, для продуктивной среды он категорически не подходит.

* _LocalExecutor_ — этот вид исполнителя даёт максимальные ощущения продуктивной среды в тестовом окружении (или окружении разработки). Он умеет выполнять задачи параллельно (например, исполнять несколько DAGов одновременно) путём порождения дочерних процессов, но всё же не совсем предназначен для продакшена ввиду ряда проблем: ограничение при масштабировании (исполнитель этого типа ограничен ресурсами машины на котором он запущен), а также отсутствие отказоустойчивости (если машина с этим типом воркера падает, то задачи перестают исполнять до момента её возвращения в строй). При небольшом количестве задач всё же можно использовать LocalExecutor, т.к. это проще, быстрее и не требует настройки дополнительных сервисов.

* _CeleryExecutor_ — наиболее популярный вид исполнения задач, позволяет иметь несколько воркеров, работающих на разных машинах. Под капотом использует всю магию таск-менеджера Celery, а соответственно тянет за собой все зависимости этого инструмента. Чтобы использовать CeleryExecutor необходимо дополнительно настроить брокер сообщений. Чаще всего используют либо Redis, либо RabbitMQ. Преимущества этого вида в том, что его легко масштабировать — поднял новую машину с воркером, и он готов выполнять требуемую работу, а также в отказоустойчивости. В случае падения одного из воркеров его работа будет передана любому из живых.

* _DaskExecutor_ — на базе Dask – библиотеки параллельных вычислений на языке Python, которая может масштабироваться на кластер из 100 узлов, поддерживая методы машинного обучения с помощью технологий NumPy/Pandas/Lists, а также цепочки задач в виде DAG-графов. Как и Celery, Dask самостоятельно не поддерживает очереди. Поэтому, если задача Airflow была создана с очередью, об этом будет выдано предупреждение, а сама задача будет отправлена в кластер.

* _KubernetesExecutor_ — относительно новый вид исполнения задач на кластере Kubernetes. Задачи исполняются как новые pod инстансы. В связи с развитием контейнеров и их повсеместным использованием, данный вид исполнения может быть интересен широкому кругу людей. Но у него есть минус — если у вас нет Kubernetes кластера, то настроить его будет непростым упражнением.

###  Принципы работы и архитектура

После определения цепочки задач в виде DAG-графа, последовательность их выполнения можно детализировать следующим образом:

1. СУБД с метаданными, например, PostgreSQL, хранит записи обо всех задачах DAG-графа и их статусе (в очереди, запланировано, выполняется, успешно выполнено, не выполнено и пр);
2. планировщик (Sheduler) читает метаданные из этой СУБД, чтобы проверить состояние каждой задачи и решить, что и в каком порядке нужно сделать;
3. планировщик передает информацию исполнителю (Executor), чтобы тот выделил ресурсы для фактического выполнения задач по мере их постановки в очередь.

Планировщик запускает экземпляр исполнителя, указанного в конфигурационном файле airflow.cfg. Разница между исполнителями сводится к ресурсам, которые у них есть, и к тому, как они будут использовать эти ресурсы для распределения работ. В частности, при работе с LocalExecutor задачи будут выполняться как локальные подпроцессы. В других случаях – задачи выполняются удаленно, на разных рабочих узлах (worker’ах). 

Более подробно об основных концепциях Airflow можно также почитать в [официальной документации](https://airflow.apache.org/docs/apache-airflow/stable/concepts.html).

Познакомившись с основными логическими компонентами, которые обеспечивают полноценную работу Airflow, рассмотрим теперь архитектуру инструмента в целом. Архитектурную схему Airflow можно представить в следующем виде:

<img src="images/1/airflow_basic_architecture.png" style="width: 400px;"> 

### Summary

Итак, Airflow позволяет определять DAG, показывая взаимозависимость входящих в него задач. Далее эти задачи запускаются по нужному расписанию в правильном порядке с отслеживанием прогресса и отправкой уведомлений о возникших сбоях. Например, в состав data pipeline’а могут входить такие задачи, как считывание сообщений из топиков Apache Kafka, их обогащение историческими данными из HDFS с помощью Spark или Hive. При этом периодичность и порядок запуска задач может изменяться в зависимости от бизнес-логики. С учетом популяризации DataOps-идей, суть любого инструмента оркестровки сводится к обеспечению централизованных, повторяемых, воспроизводимых и эффективных рабочих процессов. Таким образом, средство workflow-оркестрации выступает в качестве единого центра управления для всех автоматизированных задач.

Data Engineer сталкивается с необходимостью создания собственных конвейеров доставки и обработки данных (data pipeline) с помощью специальных ETL-фреймворков. При выборе такого инструмента для Big Data стоит помнить про 2 типа обработки данных: потоковую (stream) и пакетную (batch). С задачами непрерывной маршрутизации и доставки потоковых данных успешно справляются другие инструменты, например, Apache Kafka, которая будет рассмотрена в следующих курсах. Для пакетных же Big Data pipelines предназначен Apache Airflow.

## Дополнительные материалы

1. [Формат cron](http://www.nncron.ru/nncronlt/help/RU/working/cron-format.htm)
2. [Проверка корректности cron'а](https://crontab.guru/)
3. [Официальная документация Apache Airflow](https://airflow.apache.org/docs/apache-airflow/stable/index.html)
4. [Концепции Apache Airflow](https://airflow.apache.org/docs/apache-airflow/stable/concepts.html)
5. [Airflow Executors Explained](https://www.astronomer.io/guides/airflow-executors-explained/)
6. [Airflow Tutorials on YouTube (на англ. языке)](https://www.youtube.com/watch?v=AHMm1wfGuHE&list=PLYizQ5FvN6pvIOcOd6dFZu3lQqc6zBGp2)

## Домашнее задание

__1.__ Какой из флагов утилиты crontab покажет список существующих кронов?

Ответ: crontab -l

__2.__ Напишите крон, который будет запускаться каждую пятницу в 9 часов вечера.

Ответ: <ваш ответ здесь>

__3.__ Напишите крон, который будет запускаться каждое воскресенье марта месяца на протяжении всего дня с интервалом в 4 часа (т.е. запуск будет в 2021-03-07 00:00:00, затем 2021-03-07 04:00:00 и т.д.)

Ответ: <ваш ответ здесь>

__4.__ Отметьте все картинки, где изображен направленный ациклический граф.

<img src="images/1/dag_question.png" style="width: 500px;"> 

Ответ: <ваш ответ здесь>

__5.__ Опишите своими словами, как Вы поняли, чем отличается task от operator?

Ответ: <ваш ответ здесь>